### Leer todos los datos que son requeridos

In [0]:
dbutils.widgets.text("p_file_date", "2024-12-30")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run "../includes/configuration"

In [0]:
movies_df = spark.read.parquet(f"{silver_folder_path}/movies").filter(
    f"file_date = '{v_file_date}'"
)


In [0]:
languages_df = spark.read.parquet(f"{silver_folder_path}/languages")

In [0]:
movies_laguages_df = spark.read.parquet(f"{silver_folder_path}/movies_languages").filter(
    f"file_date = '{v_file_date}'"
)

In [0]:
genres_df = spark.read.parquet(f"{silver_folder_path}/genres")

In [0]:
movies_genre_df = spark.read.parquet(f"{silver_folder_path}/movies_genres").filter(
    f"file_date = '{v_file_date}'"
)

### JOIN entre "languages" y "movies_language"

In [0]:
languages_mov_lan_df = languages_df.join(
  movies_laguages_df,
  languages_df.language_id == movies_laguages_df.language_id,
  "inner"
).select(
  languages_df.language_name,
  movies_laguages_df.movie_id
)

### JOIN entre "genres" y "movies_genres"

In [0]:
genres_mov_gen_df = genres_df.join(
    movies_genre_df,
    genres_df.genre_id == movies_genre_df.genre_id,
    "inner"
).select(
    genres_df.genre_name,
    movies_genre_df.movie_id
)

### JOIN entre "movies_df", "language_mov_lan_df" y "genres_mov_gen_df"

In [0]:
movie_filter_df = movies_df.filter("year_release_date >= 2000")

In [0]:
results_movies_genres_languages_df = movie_filter_df.join(
    languages_mov_lan_df,
    movie_filter_df.movie_id == languages_mov_lan_df.movie_id,
    "inner"
).join(
    genres_mov_gen_df,
    movie_filter_df.movie_id == genres_mov_gen_df.movie_id,
    "inner"
)

In [0]:
from pyspark.sql.functions import lit

In [0]:
results_df = results_movies_genres_languages_df.select(
    "title", "duration_time", "release_date", "vote_average", "language_name", "genre_name"
).withColumn(
    "created_date", lit(v_file_date)
)

In [0]:
results_order_by_df = results_df.orderBy(results_df.release_date.desc())


In [0]:
display(results_order_by_df)

### Escribir datos en el DataLake en formato "Parquet"

In [0]:
%run "../includes/commom_functions"

In [0]:
overwrite_partition(results_order_by_df, "movie_gold", "results_movie_genre_language", "created_date")

In [0]:
#results_order_by_df.write.mode("overwrite").parquet(f"{gold_folder_path}/results_movie_genre_language")

#results_order_by_df.write.mode("overwrite").format("parquet").saveAsTable("movie_gold.results_movie_genre_language")

results_order_by_df.write.mode("append").partitionBy("created_date").format("parquet").saveAsTable("movie_gold.results_movie_genre_language")

In [0]:
#display(spark.read.parquet(f"{gold_folder_path}/results_movie_genre_language"))


In [0]:
%sql
select * from movie_gold.results_movie_genre_language

In [0]:
%sql
select created_date, count(1) from movie_gold.results_movie_genre_language group by created_date 